In [1]:
import re
import requests
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import quote
from concurrent.futures import ThreadPoolExecutor
import time

         
##解析表格
def parse_one_page(html):
    soup = BeautifulSoup(html.text,'lxml')
    try:
        content = soup.select('table')[0] #将返回的list改为bs4类型    
        tbl = pd.read_html(content.prettify())[0]
        tbl.columns=['label','detail']
    # prettify()优化代码,[0]从pd.read_html返回的list中提取出DataFrame
    except:
        label = ['主要经营产品：','营业执照号码：','所属分类：']
        detail = ['','','']
        D = {'label': label,'detail':detail}
        tbl = pd.DataFrame(D)
    return tbl

##简介是直接获取，其他信息写进一个dataframe里
def get_detail_http(company,url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1'
        }
    resp = requests.get(url = url, headers = headers)
    resp.encoding = 'utf-8'
    html = etree.HTML(str(resp.text))    
    comp_raw = html.xpath('//*[@id="aboutuscontent"]')
    if comp_raw == []:
        comp_abstract = 'none'
    else:
        comp_abstract = html.xpath('//*[@id="aboutuscontent"]')[0].xpath('string(.)')
    comp_abstract = comp_abstract.replace('\n','').replace('\r','').strip()
   # index_1 = comp_abstract.find('联系')
    comp_info_table = parse_one_page(resp)
    return comp_abstract,comp_info_table

##整合信息 
def info_concat(company,url):
    abstract,info_table = get_detail_http(company,url)
    product = info_table.loc[info_table['label']=='主要经营产品：']['detail'].tolist()
    product =    ''.join([i for i in product if not i.isdigit()]) 
    product =  product.replace(' ','').strip()
    if product == '':
        product = 'none'
        
    number = info_table.loc[info_table['label']=='营业执照号码：']['detail'].tolist()
    number =    ''.join([i for i in number if not i.isdigit()]) 
    if number == '':
        number = 'none'
    
    category = info_table.loc[info_table['label']=='所属分类：']['detail'].tolist()
    category =    ''.join([i for i in category if not i.isdigit()]) 
    if category == '':
        category = 'none'  
    write_to_txt(company,product,number,category,abstract)

def write_to_txt(company,product,number,category,abstract):
#     product1 = product.replace('，','').replace(',','').replace('。','').replace(' ','').replace('.','').replace(':','').replace('：','').replace('（','(').replace('）',')').strip('许可项目').strip('主营项目').strip('主营产品').strip('服务').strip()[0:6]
#     abstract1= abstract.replace('\n','').replace('，','').replace('。','').replace(' ','').replace('.','').replace('、','').replace(',','').replace('；','').replace(':','').replace('：','').replace('（','(').replace('）',')').strip()
#     index = abstract1.find(product1)
#     if index == -1:
        with open('info_valid_sci_and_techt_0802.txt',mode = 'a+',encoding='utf-8') as c:  
                c.write(company + '\t')
                c.write(product + '\t')
                c.write(number + '\t')
                c.write(category + '\t')
                c.write(abstract + '\n')
                c.close() 
                #print(company)
#     else:
#         pass

In [11]:
if __name__ == '__main__':
    i = 0
    with open('/Users/wangchenghang/Desktop/tencent/sci_and_tech_final_out.txt',"r") as f:
        for line in f.readlines()[170000:]:
            i += 1
            line = line.strip('\n')
            company = line.split(sep = '\t')[0]
            url = line.split(sep = '\t')[1]
            info_concat(company,url)
            if i%50 == 0:
                print(i)
                print(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))

50
2021-08-02 14:40:41
100
2021-08-02 14:40:57
150
2021-08-02 14:41:11
200
2021-08-02 14:41:31
250
2021-08-02 14:41:45
300
2021-08-02 14:41:59
350
2021-08-02 14:42:12
400
2021-08-02 14:42:25
450
2021-08-02 14:42:39
500
2021-08-02 14:42:55
550
2021-08-02 14:43:08
600
2021-08-02 14:43:27
650
2021-08-02 14:43:40
700
2021-08-02 14:43:54
750
2021-08-02 14:44:08
800
2021-08-02 14:44:21
850
2021-08-02 14:44:36
900
2021-08-02 14:44:50
950
2021-08-02 14:45:03
1000
2021-08-02 14:45:17
1050
2021-08-02 14:45:30
1100
2021-08-02 14:45:45
1150
2021-08-02 14:46:07
1200
2021-08-02 14:46:23
1250
2021-08-02 14:46:36
1300
2021-08-02 14:46:50
1350
2021-08-02 14:47:07
1400
2021-08-02 14:47:20
1450
2021-08-02 14:47:35
1500
2021-08-02 14:47:50
1550
2021-08-02 14:48:04
1600
2021-08-02 14:48:25
1650
2021-08-02 14:48:38
1700
2021-08-02 14:48:52
1750
2021-08-02 14:49:08
1800
2021-08-02 14:49:28
1850
2021-08-02 14:49:48
1900
2021-08-02 14:50:07
1950
2021-08-02 14:50:29
2000
2021-08-02 14:50:48
2050
2021-08-02 14:5

16250
2021-08-02 16:04:53
16300
2021-08-02 16:05:06
16350
2021-08-02 16:05:26
16400
2021-08-02 16:05:45
16450
2021-08-02 16:06:00
16500
2021-08-02 16:06:14
16550
2021-08-02 16:06:30
16600
2021-08-02 16:06:44
16650
2021-08-02 16:06:57
16700
2021-08-02 16:07:11
16750
2021-08-02 16:07:25
16800
2021-08-02 16:07:39
16850
2021-08-02 16:07:53
16900
2021-08-02 16:08:08
16950
2021-08-02 16:08:21
17000
2021-08-02 16:08:35
17050
2021-08-02 16:08:48
17100
2021-08-02 16:09:05
17150
2021-08-02 16:09:24
17200
2021-08-02 16:09:38
17250
2021-08-02 16:09:54
17300
2021-08-02 16:10:11
17350
2021-08-02 16:10:25
17400
2021-08-02 16:10:39
17450
2021-08-02 16:10:52
17500
2021-08-02 16:11:05
17550
2021-08-02 16:11:18
17600
2021-08-02 16:11:32
17650
2021-08-02 16:11:45
17700
2021-08-02 16:11:58
17750
2021-08-02 16:12:12
17800
2021-08-02 16:12:31
17850
2021-08-02 16:12:48
17900
2021-08-02 16:13:01
17950
2021-08-02 16:13:18
18000
2021-08-02 16:13:32
18050
2021-08-02 16:13:51
18100
2021-08-02 16:14:05
18150
2021-0

32050
2021-08-02 17:26:14
32100
2021-08-02 17:26:27
32150
2021-08-02 17:26:42
32200
2021-08-02 17:26:55
32250
2021-08-02 17:27:08
32300
2021-08-02 17:27:22
32350
2021-08-02 17:27:35
32400
2021-08-02 17:27:48
32450
2021-08-02 17:28:02
32500
2021-08-02 17:28:15
32550
2021-08-02 17:28:29
32600
2021-08-02 17:28:42
32650
2021-08-02 17:28:56
32700
2021-08-02 17:29:09
32750
2021-08-02 17:29:22
32800
2021-08-02 17:29:38
32850
2021-08-02 17:29:51
32900
2021-08-02 17:30:04
32950
2021-08-02 17:30:18
33000
2021-08-02 17:30:38
33050
2021-08-02 17:30:54
33100
2021-08-02 17:31:09
33150
2021-08-02 17:31:23
33200
2021-08-02 17:31:36
33250
2021-08-02 17:31:50
33300
2021-08-02 17:32:10
33350
2021-08-02 17:32:26
33400
2021-08-02 17:32:39
33450
2021-08-02 17:32:52
33500
2021-08-02 17:33:06
33550
2021-08-02 17:33:18
33600
2021-08-02 17:33:32
33650
2021-08-02 17:33:45
33700
2021-08-02 17:33:58
33750
2021-08-02 17:34:12
33800
2021-08-02 17:34:26
33850
2021-08-02 17:34:39
33900
2021-08-02 17:34:52
33950
2021-0

In [8]:
i

26031

In [9]:
53766+1631+26031

81428